In [1]:
from transformer_lens import HookedTransformer
import transformer_lens
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import transformer_lens.utils as utils
import hashlib
import yaml 
import hashlib
import pickle
import numpy as np
import matplotlib.pyplot as plt 

/root/mech_interp_research/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = utils.get_device()

reference_model_path = 'meta-llama/Llama-3.1-8B'
baseline_model_path = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

baseline_model_hf = AutoModelForCausalLM.from_pretrained(baseline_model_path, torch_dtype=torch.bfloat16)
baseline_model_tokenizer = AutoTokenizer.from_pretrained(baseline_model_path)

baseline_model = HookedTransformer.from_pretrained_no_processing(
    reference_model_path,
    hf_model=baseline_model_hf,
    tokenizer=baseline_model_tokenizer,
    device=device,
    move_to_device=True,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  6.19it/s]


Loaded pretrained model meta-llama/Llama-3.1-8B into HookedTransformer


In [3]:
model=baseline_model

In [8]:
model([1,2,3], 'logits')

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [17]:
torch.concatenate([model.to_tokens("hi"), torch.tensor([[11,]], device='cuda')], -1)

tensor([[128000,   6151,     11]], device='cuda:0')

In [ ]:

def compute_feature_mode_metric(
    model: HookedTransformer,
    prompt: str,
    pos_features: list[list[int]],
    neg_features: list[list[int]],
):
    """
    Computes:
      - normalized probabilities for each feature sequence (sum to 1)
      - unnormalized log-scores for each
      - logit (log-odds) of producing any negative feature.
    Uses model.to_tokens() for tokenization.
    """
    device = model.cfg.device
    # 1) Tokenize prompt with the built-in hook
    #    (adds BOS if the model is configured to)
    input_ids = model.to_tokens(prompt).to(device)

    # 2) Score one feature sequence by accumulating log-probs
    def sequence_score(feature: list[int]) -> torch.Tensor:
        ctx = input_ids.clone()
        total_log_prob = torch.tensor(0.0, device=device)
        for tok in feature:
            # run the model on the current context
            logits, activations = model.run_with_cache(ctx) #turn off this 
            last_logits = logits[:, -1, :]  # [1, vocab_size]
            log_probs = torch.log_softmax(last_logits, dim=-1)
            total_log_prob = total_log_prob + log_probs[0, tok]
            # append the ground-truth token to the context
            ctx = torch.cat([ctx, torch.tensor([[tok]], device=device)], dim=1)
        return total_log_prob

    # 3) Compute scores for all features
    all_features = pos_features + neg_features
    scores = torch.stack([sequence_score(f) for f in all_features])  # (n+m,)

    # 4) Softmax to get normalized probabilities
    norm_probs = torch.softmax(scores, dim=0)                        # (n+m,)

    # 5) Sum up the negative-feature mass & compute logit
    num_pos   = len(pos_features)
    neg_prob  = norm_probs[num_pos:].sum()
    neg_logit = torch.log(neg_prob) - torch.log(1 - neg_prob)

    return norm_probs, scores, neg_logit




In [5]:
pos_features = [[1271], [1271, 1505], [1271, 8417], [334, 37942, 25]]
neg_features = [[33413]]

for pf in pos_features:
    print(model.to_string(pf))

print("\n\n")

for nf in neg_features:
    print(model.to_string(nf))

To
To find
To determine
**Solution:



Okay


In [6]:

prompt       = "<｜User｜>What is 8/4<｜Assistant｜><think>\n"
pos_features = [[1271], [1271, 1505], [1271, 8417], [334, 37942, 25]]
neg_features = [[33413]]

norm_probs, scores, neg_logit = compute_feature_mode_metric(
    model, prompt, pos_features, neg_features
)

print("Normalized probabilities:", norm_probs)
print("Unnormalized log-scores:", scores)
print("Negative-feature logit:", neg_logit)

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 44.34 GiB of which 24.81 MiB is free. Process 2410955 has 44.31 GiB memory in use. Of the allocated memory 43.97 GiB is allocated by PyTorch, and 32.28 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

prompt       = "<｜User｜>What is 8/4<｜Assistant｜><think>\nOkay, I know 8/4=2\n</think>s"
pos_features = [[1271], [1271, 1505], [1271, 8417], [334, 37942, 25]]
neg_features = [[33413]]

norm_probs, scores, neg_logit = compute_feature_mode_metric(
    model, prompt, pos_features, neg_features
)

print("Normalized probabilities:", norm_probs)
print("Unnormalized log-scores:", scores)
print("Negative-feature logit:", neg_logit)